# 🌍 AI Travel Agent with Microsoft Agent Framework (.NET)

## 📋 Scenario Overview

This notebook demonstrates how to build an intelligent travel planning agent using the Microsoft Agent Framework for .NET. The agent can automatically generate personalized day-trip itineraries for random destinations around the world.

**Key Capabilities:**
- 🎲 **Random Destination Selection**: Uses a custom tool to pick vacation spots
- 🗺️ **Intelligent Trip Planning**: Creates detailed day-by-day itineraries
- 🔄 **Real-time Streaming**: Supports both immediate and streaming responses
- 🛠️ **Custom Tool Integration**: Demonstrates how to extend agent capabilities

## 🔧 Technical Architecture

### Core Technologies
- **Microsoft Agent Framework**: Latest .NET implementation for AI agent development
- **GitHub Models Integration**: Uses GitHub's AI model inference service
- **OpenAI API Compatibility**: Leverages OpenAI client libraries with custom endpoints
- **Secure Configuration**: Environment-based API key management

### Key Components
1. **AIAgent**: The main agent orchestrator that handles conversation flow
2. **Custom Tools**: `GetRandomDestination()` function available to the agent
3. **Chat Client**: GitHub Models-backed conversation interface
4. **Streaming Support**: Real-time response generation capabilities

### Integration Pattern
```mermaid
graph LR
    A[User Request] --> B[AI Agent]
    B --> C[GitHub Models API]
    B --> D[GetRandomDestination Tool]
    C --> E[Travel Itinerary]
    D --> E
```

## 🚀 Getting Started

**Prerequisites:**
- .NET 9.0 or higher
- GitHub Models API access token
- Environment variables configured in `.env` file

**Required Environment Variables:**
```env
GITHUB_TOKEN=your_github_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Run the cells below in sequence to see the travel agent in action!

In [1]:
// 📦 NuGet Package Reference: Microsoft Extensions AI
// This provides the core AI abstraction layer for .NET applications
// Essential for building AI agents with standardized interfaces
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
// 🔗 Local Reference: OpenAI Agent Extensions
// This references the local build of the Microsoft Extensions AI Agents OpenAI library
// Contains OpenAI-specific implementations for the agent framework
// Required for integrating with OpenAI models and GitHub Models
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0\Microsoft.Agents.AI.OpenAI.dll"

In [3]:
// 🤖 Local Reference: Core Agent Framework
// This references the core Microsoft Extensions AI Agents library
// Provides the fundamental agent abstractions and base classes
// Essential for creating and managing AI agents
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [4]:
// 🔌 NuGet Package Reference: OpenAI Integration
// Preview version of Microsoft Extensions AI OpenAI integration
// Provides OpenAI client and service implementations
// Enables communication with OpenAI API and GitHub Models
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [5]:
// 🔧 NuGet Package Reference: Environment Variables
// DotNetEnv allows loading environment variables from .env files
// Essential for managing API keys and configuration securely
// Helps keep sensitive information out of source code
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
// 📚 Import Required Namespaces
using System;                          // Core system functionality and base types
using System.ComponentModel;           // For Description attributes on tool functions
using System.ClientModel;             // Client model abstractions for API interactions

using Microsoft.Extensions.AI;        // Core AI abstractions and interfaces
using Microsoft.Agents.AI;           // Agent framework classes and interfaces
using OpenAI;                         // OpenAI client library for API communication

In [7]:
// 🔧 Import Environment Variable Loading
// DotNetEnv namespace for loading .env files
// Enables secure configuration management from external files
using DotNetEnv;

In [8]:
// 🔧 Load Environment Variables from .env File
// This loads configuration from a .env file located 3 directories up
// Required variables: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
// Ensures secure handling of API credentials and configuration
Env.Load("../../../.env");

In [9]:
// 🎲 Tool Function: Random Destination Generator
// This static method will be available to the agent as a callable tool
// The [Description] attribute helps the AI understand when to use this function
// This demonstrates how to create custom tools for AI agents
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    // List of popular vacation destinations around the world
    // The agent will randomly select from these options
    var destinations = new List<string>
    {
        "Paris, France",         // European cultural capital
        "Tokyo, Japan",          // Asian modern metropolis
        "New York City, USA",    // American urban experience
        "Sydney, Australia",     // Oceanic coastal beauty
        "Rome, Italy",           // Historical European city
        "Barcelona, Spain",      // Mediterranean cultural hub
        "Cape Town, South Africa", // African scenic destination
        "Rio de Janeiro, Brazil",  // South American beach city
        "Bangkok, Thailand",        // Southeast Asian cultural center
        "Vancouver, Canada"         // North American natural beauty
    };

    // Generate random index and return selected destination
    // Uses System.Random for simple random selection
    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
// 🔑 Extract Configuration from Environment Variables
// Retrieve GitHub Models API endpoint, throws exception if not set
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
// Get the model ID, defaults to gpt-4o-mini if not specified
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
// Retrieve GitHub token for authentication, throws exception if not set
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
// ⚙️ Configure OpenAI Client Options
// Create configuration options to point to GitHub Models endpoint
// This redirects OpenAI client calls to GitHub's model inference service
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)  // Set custom endpoint for GitHub Models
};

In [12]:
// 🔌 Initialize OpenAI Client with GitHub Models Configuration
// Create OpenAI client using GitHub token for authentication
// Configure it to use GitHub Models endpoint instead of OpenAI directly
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
// 🤖 Create AI Agent with Travel Planning Capabilities
// Initialize OpenAI client, get chat client for specified model, and create AI agent
// Configure agent with travel planning instructions and random destination tool
// The agent can now plan trips using the GetRandomDestination function
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions)
    .GetChatClient(github_model_id)  // Get chat client for the specified model
    .CreateAIAgent(
        instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", 
        tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]  // Register the tool function
    );

In [14]:
// 🚀 Execute Agent: Plan a Day Trip (Non-Streaming)
// Run the agent with a travel planning request
// The agent will use the GetRandomDestination tool and create a complete itinerary
// Returns the full response once processing is complete
Console.WriteLine(await agent.RunAsync("Plan me a day trip"));

I have a great destination for your day trip: **Barcelona, Spain**! Here’s a suggested itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional Spanish breakfast. Enjoy "tostada con tomate" (toasted bread with tomato) and a coffee at a local café.
- **Visit La Sagrada Familia**: Head to the iconic basilica designed by Antoni Gaudí. Take some time to explore the stunning architecture and perhaps take a guided tour.

### Afternoon
- **Lunch at a Tapas Bar**: Enjoy a leisurely lunch at a tapas bar. Try different dishes like patatas bravas, jamón ibérico, and gambas al ajillo (garlic shrimp).
- **Explore Park Güell**: After lunch, visit this famous park also designed by Gaudí. Stroll through the colorful mosaics and enjoy breathtaking views of the city.

### Evening
- **Wander Through Gothic Quarter**: Spend your evening exploring the quaint streets of the Gothic Quarter. Check out the historic architecture and small shops.
- **Dinner at a Res

In [15]:
// 🌊 Execute Agent: Plan a Day Trip (Streaming Response)
// Run the agent with streaming enabled for real-time response display
// Shows the agent's thinking and response as it generates the content
// Provides better user experience with immediate feedback
await foreach (var update in agent.RunStreamingAsync("Plan me a day trip"))
{
    Console.Write(update);  // Display each part of the response as it arrives
}

How about a day trip to Paris, France? Here’s a suggested itinerary:

### Morning
- **Breakfast at a Café**: Start your day with a traditional French breakfast at a local café. Enjoy croissants, pain au chocolat, and a cup of coffee.
- **Visit the Eiffel Tower**: Head over to the Eiffel Tower. You can go up for stunning views of the city, or simply enjoy the view from the park below.

### Midday
- **Lunch at a Bistro**: Try a classic French bistro for lunch. Options include a croque monsieur or a salade niçoise.
- **Louvre Museum**: Spend the afternoon exploring the Louvre. Don’t miss the Mona Lisa and other masterpieces.

### Afternoon
- **Stroll along the Seine**: Take a leisurely walk along the Seine River. Stop at the bookstalls and enjoy the scenic views.
- **Visit Notre-Dame Cathedral**: Although it’s under restoration, you can still appreciate the architecture from the outside.

### Evening
- **Dinner in Montmartre**: Head to the bohemian neighborhood of Montmartre for dinner. E